In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def rot_aut(Xtr,ytr):
    # aug on train
    data_cnt = len(ytr)
    print(data_cnt)
    aug_X = []
    aug_y = []

    for i in range(data_cnt):
        img = Xtr[i]
        tmp_y = ytr[i]

        # org img
        aug_X.append(img)
        aug_y.append(tmp_y)

        # flip
        tmp_img = np.fliplr(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.flipud(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.rot90(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)
    return np.array(aug_X),np.array(aug_y)

In [11]:
from keras.callbacks import ModelCheckpoint
from keras.layers import Input,AveragePooling2D,GlobalMaxPooling2D,GlobalAveragePooling2D,Flatten
from keras import layers
from keras.models import Model

def conv2d_bn(x,
              filters,
              num_row,
              num_col,
              padding='same',
              strides=(1, 1),
              name=None):

    if name is not None:
        bn_name = name + '_bn'
        conv_name = name + '_conv'
    else:
        bn_name = None
        conv_name = None
    bn_axis = 3
    x = Conv2D(
        filters, (num_row, num_col),
        strides=strides,
        padding=padding,
        use_bias=False,
        name=conv_name)(x)
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    x = Activation('relu', name=name)(x)
    return x

def create_model():
    img_input = Input(shape=(75,75,3))
    channel_axis = 3
    
    # bn
    x = conv2d_bn(img_input, 32, 3, 3, strides=(2, 2), padding='valid')
    x = conv2d_bn(x, 32, 3, 3, padding='valid')
    x = conv2d_bn(x, 64, 3, 3)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv2d_bn(x, 64, 1, 1, padding='valid')
    x = conv2d_bn(x, 64, 3, 3, padding='valid')
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    # mixed 0, 1, 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')
    
    # mixed 1: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')
    
    # mixed 2: 35 x 35 x 256
    branch1x1 = conv2d_bn(x, 64, 1, 1)

    branch5x5 = conv2d_bn(x, 48, 1, 1)
    branch5x5 = conv2d_bn(branch5x5, 64, 5, 5)

    branch3x3dbl = conv2d_bn(x, 64, 1, 1)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)
    branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 3)

    branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = conv2d_bn(branch_pool, 64, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')

    x = Conv2D(64, 3, strides=2, padding='same',activation='relu')(x)
    x = Flatten()(x)
    x = Dense(256,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(img_input, x, name='inception_v3')
print('model model')

test_m = create_model()
test_m.summary()

model model
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
conv2d_159 (Conv2D)             (None, 37, 37, 32)   864         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_157 (BatchN (None, 37, 37, 32)   96          conv2d_159[0][0]                 
__________________________________________________________________________________________________
activation_157 (Activation)     (None, 37, 37, 32)   0           batch_normalization_157[0][0]    
_________________________________________________________________________________________________

In [12]:
from sklearn.model_selection import KFold
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        curr_x,curr_y = rot_aut(curr_x,curr_y)
        val_x,val_y = train_x[test_index],y[test_index]
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        model.fit(curr_x,curr_y,
                  validation_data=(val_x,val_y),
                  batch_size=32, epochs=20, 
                  verbose=2,
                  callbacks=[model_chk]
                 )
        model = load_model(model_p)
        
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

1203
Train on 4812 samples, validate on 401 samples
Epoch 1/20
Epoch 00001: val_loss improved from inf to 1.30480, saving model to best_m.h5
 - 21s - loss: 0.4451 - acc: 0.7833 - val_loss: 1.3048 - val_acc: 0.5087
Epoch 2/20
Epoch 00002: val_loss did not improve
 - 9s - loss: 0.3178 - acc: 0.8554 - val_loss: 3.0415 - val_acc: 0.5087
Epoch 3/20
Epoch 00003: val_loss improved from 1.30480 to 1.13509, saving model to best_m.h5
 - 10s - loss: 0.2693 - acc: 0.8778 - val_loss: 1.1351 - val_acc: 0.5112
Epoch 4/20
Epoch 00004: val_loss improved from 1.13509 to 0.33969, saving model to best_m.h5
 - 10s - loss: 0.2497 - acc: 0.8857 - val_loss: 0.3397 - val_acc: 0.8529
Epoch 5/20
Epoch 00005: val_loss improved from 0.33969 to 0.31383, saving model to best_m.h5
 - 10s - loss: 0.2246 - acc: 0.9059 - val_loss: 0.3138 - val_acc: 0.8379
Epoch 6/20
Epoch 00006: val_loss improved from 0.31383 to 0.24113, saving model to best_m.h5
 - 9s - loss: 0.1923 - acc: 0.9165 - val_loss: 0.2411 - val_acc: 0.9002
Ep

 - 38s - loss: 0.4073 - acc: 0.8026 - val_loss: 1.1951 - val_acc: 0.5062
Epoch 2/20
Epoch 00002: val_loss improved from 1.19509 to 0.66883, saving model to best_m.h5
 - 9s - loss: 0.2950 - acc: 0.8587 - val_loss: 0.6688 - val_acc: 0.5087
Epoch 3/20
Epoch 00003: val_loss improved from 0.66883 to 0.51426, saving model to best_m.h5
 - 10s - loss: 0.2474 - acc: 0.8867 - val_loss: 0.5143 - val_acc: 0.7456
Epoch 4/20
Epoch 00004: val_loss improved from 0.51426 to 0.35914, saving model to best_m.h5
 - 10s - loss: 0.2300 - acc: 0.8971 - val_loss: 0.3591 - val_acc: 0.8304
Epoch 5/20
Epoch 00005: val_loss did not improve
 - 9s - loss: 0.1935 - acc: 0.9194 - val_loss: 0.4309 - val_acc: 0.8429
Epoch 6/20
Epoch 00006: val_loss did not improve
 - 9s - loss: 0.1880 - acc: 0.9196 - val_loss: 0.4407 - val_acc: 0.8055
Epoch 7/20
Epoch 00007: val_loss did not improve
 - 10s - loss: 0.1543 - acc: 0.9296 - val_loss: 0.5825 - val_acc: 0.7980
Epoch 8/20
Epoch 00008: val_loss did not improve
 - 10s - loss: 0.

In [13]:
import pickle
with open('../features/incept_1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))

submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/incept_1_sub.csv', index=False)

0.303490242014
         id  is_iceberg
0  5941774d    0.413197
1  4023181e    0.433543
2  b20200e4    0.179360
3  e7f018bb    0.995098
4  4371c8c3    0.333573
